# Quantum Portfolio Optimization: Classical Approaches

This notebook explores classical approaches to portfolio optimization as a baseline for comparison with quantum methods.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import time

# Add parent directory to path
sys.path.append(os.path.abspath('..'))

from portfolio.markowitz import MarkowitzOptimizer
from portfolio.constraints import PortfolioConstraints
from portfolio.efficient_frontier import EfficientFrontier
from portfolio.risk_measures import RiskMeasures
from utilities.data_preparation import DataPreparation
from utilities.visualization import PortfolioVisualization
from benchmarks.classical_solvers import ClassicalPortfolioSolvers
from benchmarks.test_cases import TestCases

## 1. Load Sample Financial Data

In [ ]:
# Option 1: Generate random data
random_data = DataPreparation.generate_random_data(n_assets=10, n_periods=252, seed=42)
print("Random data generated for 10 assets over 252 trading days")

# Extract components
cov_matrix = random_data['covariance_matrix']
exp_returns = random_data['expected_returns']

# Option 2: Use test case
balanced_portfolio = TestCases.balanced_portfolio()
print(f"\nTest case: {balanced_portfolio['name']}")
print(f"Description: {balanced_portfolio['description']}")
print(f"Assets: {balanced_portfolio['asset_names']}")

## 2. Markowitz Mean-Variance Optimization

The foundational Markowitz portfolio optimization seeks to minimize portfolio risk for a given level of expected return.

In [ ]:
# Use the balanced portfolio test case
cov_matrix = balanced_portfolio['covariance_matrix'].values
exp_returns = balanced_portfolio['expected_returns'].values
asset_names = balanced_portfolio['asset_names']

# Initialize Markowitz optimizer
markowitz_optimizer = MarkowitzOptimizer(cov_matrix, exp_returns)

# Set target return (e.g., average of expected returns)
target_return = np.mean(exp_returns)
print(f"Target return: {target_return:.4f}")

# Run optimization
start_time = time.time()
result = markowitz_optimizer.optimize(target_return=target_return, allow_short_selling=False)
end_time = time.time()

print(f"\nOptimization completed in {end_time - start_time:.4f} seconds")
print(f"Portfolio return: {result['portfolio_return']:.4f}")
print(f"Portfolio risk: {result['portfolio_risk']:.4f}")
print(f"Sharpe ratio: {result['sharpe_ratio']:.4f}")

# Plot portfolio weights
plt.figure(figsize=(10, 6))
plt.bar(asset_names, result['optimal_weights'])
plt.title('Optimal Portfolio Weights')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Efficient Frontier

The efficient frontier represents the set of optimal portfolios that offer the highest expected return for a defined level of risk.

In [ ]:
# Calculate efficient frontier
start_time = time.time()
frontier = markowitz_optimizer.efficient_frontier(n_points=50, allow_short_selling=False)
end_time = time.time()

print(f"Efficient frontier calculated in {end_time - start_time:.4f} seconds")

# Plot efficient frontier
plt.figure(figsize=(10, 6))
plt.plot(frontier['frontier_risks'], frontier['frontier_returns'], 'o-')
plt.xlabel('Risk (Standard Deviation)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True, alpha=0.3)

# Add current portfolio
plt.scatter([result['portfolio_risk']], [result['portfolio_return']], 
            color='red', marker='*', s=200, label='Current Portfolio')

# Add min risk and max Sharpe portfolios
min_risk_idx = np.argmin(frontier['frontier_risks'])
plt.scatter([frontier['frontier_risks'][min_risk_idx]], [frontier['frontier_returns'][min_risk_idx]], 
            color='green', marker='o', s=100, label='Min Risk')

# Maximize Sharpe ratio
sharpe_optimizer = markowitz_optimizer.optimize_sharpe(risk_free_rate=0.0, allow_short_selling=False)
plt.scatter([sharpe_optimizer['portfolio_risk']], [sharpe_optimizer['portfolio_return']], 
            color='purple', marker='s', s=100, label='Max Sharpe')

plt.legend()
plt.tight_layout()
plt.show()

## 4. Portfolio Optimization with Cardinality Constraints

Cardinality constraints limit the number of assets in the portfolio, making the problem NP-hard.

In [ ]:
# Create a larger test case
large_portfolio = TestCases.generate_random_portfolio(n_assets=15, seed=42)
cov_matrix_large = large_portfolio['covariance_matrix'].values
exp_returns_large = large_portfolio['expected_returns'].values

# Set target return
target_return = np.mean(exp_returns_large)
print(f"Target return: {target_return:.4f}")

# Set maximum number of assets
max_assets = 5
print(f"Maximum number of assets: {max_assets}")

# Run optimization with cardinality constraint
start_time = time.time()
result = ClassicalPortfolioSolvers.solve_with_cardinality_constraint(
    cov_matrix_large,
    exp_returns_large,
    target_return,
    max_assets
)
end_time = time.time()

print(f"\nOptimization completed in {end_time - start_time:.4f} seconds")
print(f"Portfolio return: {result['portfolio_return']:.4f}")
print(f"Portfolio risk: {result['portfolio_risk']:.4f}")
print(f"Sharpe ratio: {result['sharpe_ratio']:.4f}")
print(f"Number of assets selected: {len([w for w in result['weights'] if w > 1e-6])}")

# Plot portfolio weights
weights = result['weights']
asset_labels = [f"Asset {i+1}" for i in range(len(weights))]

plt.figure(figsize=(12, 6))
plt.bar(asset_labels, weights)
plt.title('Optimal Portfolio Weights with Cardinality Constraint')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Risk Measures Beyond Variance

Modern portfolio optimization often uses risk measures beyond variance, such as Value-at-Risk (VaR) and Conditional Value-at-Risk (CVaR).

In [ ]:
# Generate random returns data
np.random.seed(42)
returns_data = np.random.normal(loc=0.001, scale=0.02, size=(252, 5))

# Portfolio weights
weights = result['weights'][:5]
weights = weights / np.sum(weights)  # Normalize to ensure sum is 1

# Calculate risk measures
var_95 = RiskMeasures.value_at_risk(weights, returns_data, confidence_level=0.95)
cvar_95 = RiskMeasures.conditional_value_at_risk(weights, returns_data, confidence_level=0.95)
drawdown = RiskMeasures.drawdown(weights, returns_data)
sortino = RiskMeasures.sortino_ratio(weights, returns_data, exp_returns[:5])

print(f"Value-at-Risk (95%): {var_95:.4f}")
print(f"Conditional Value-at-Risk (95%): {cvar_95:.4f}")
print(f"Maximum Drawdown: {drawdown['max_drawdown']:.4f}")
print(f"Sortino Ratio: {sortino:.4f}")

# Plot drawdown
plt.figure(figsize=(10, 6))
plt.plot(drawdown['drawdown_series'])
plt.axhline(y=drawdown['max_drawdown'], color='r', linestyle='--', label=f"Max Drawdown: {drawdown['max_drawdown']:.4f}")
plt.title('Portfolio Drawdown')
plt.xlabel('Time')
plt.ylabel('Drawdown')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Sector Constraints

Sector constraints limit exposure to specific sectors or asset classes.

In [ ]:
# Use sector portfolio test case
sector_portfolio = TestCases.sector_portfolio()
cov_matrix = sector_portfolio['covariance_matrix'].values
exp_returns = sector_portfolio['expected_returns'].values
asset_names = sector_portfolio['asset_names']
sectors = sector_portfolio['sectors']
sector_mapping = sector_portfolio['sector_mapping']
sector_constraints = sector_portfolio['sector_constraints']

print(f"Sectors: {set(sectors)}")
print(f"Minimum sector allocation: {sector_constraints['min_sector']}")
print(f"Maximum sector allocation: {sector_constraints['max_sector']}")

# Run optimization with sector constraints (simplified approach)
from scipy.optimize import minimize

def objective(weights):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

# Initial guess (equal weights)
initial_weights = np.ones(len(exp_returns)) / len(exp_returns)

# Constraints
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Budget constraint
]

# Add sector constraints
for sector, min_weight in sector_constraints['min_sector'].items():
    constraints.append({
        'type': 'ineq',
        'fun': lambda x, s=sector: sum(x[i] for i in range(len(x)) if sector_mapping.get(i) == s) - min_weight
    })

for sector, max_weight in sector_constraints['max_sector'].items():
    constraints.append({
        'type': 'ineq',
        'fun': lambda x, s=sector: max_weight - sum(x[i] for i in range(len(x)) if sector_mapping.get(i) == s)
    })

# Bounds (non-negative weights)
bounds = [(0, 1) for _ in range(len(exp_returns))]

# Run optimization
start_time = time.time()
result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
end_time = time.time()

weights = result.x
portfolio_return = np.dot(weights, exp_returns)
portfolio_risk = np.sqrt(objective(weights))

print(f"\nOptimization completed in {end_time - start_time:.4f} seconds")
print(f"Portfolio return: {portfolio_return:.4f}")
print(f"Portfolio risk: {portfolio_risk:.4f}")
print(f"Sharpe ratio: {portfolio_return / portfolio_risk:.4f}")

# Calculate sector allocation
sector_allocation = {}
for i in range(len(weights)):
    sector = sector_mapping.get(i)
    if sector is not None:
        sector_allocation[sector] = sector_allocation.get(sector, 0) + weights[i]

print("\nSector allocation:")
for sector_id, allocation in sector_allocation.items():
    sector_name = list(set(sectors))[sector_id]
    min_alloc = sector_constraints['min_sector'].get(sector_id, 0)
    max_alloc = sector_constraints['max_sector'].get(sector_id, 1)
    print(f"{sector_name}: {allocation:.4f} (min: {min_alloc}, max: {max_alloc})")

# Plot weights by sector
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
sector_colors = {sector: colors[i % len(colors)] for i, sector in enumerate(set(sectors))}

plt.figure(figsize=(12, 6))
bars = plt.bar(asset_names, weights)

for i, bar in enumerate(bars):
    bar.set_color(sector_colors[sectors[i]])

plt.title('Optimal Portfolio Weights with Sector Constraints')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)

# Add legend for sectors
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=sector_colors[sector], label=sector) for sector in set(sectors)]
plt.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

## 7. Computational Challenges

Classical portfolio optimization faces several computational challenges:

1. **Curse of Dimensionality**: As the number of assets increases, the complexity grows rapidly
2. **Discrete Constraints**: Cardinality constraints make the problem NP-hard
3. **Non-convex Objectives**: Advanced risk measures often lead to non-convex problems
4. **Parameter Uncertainty**: Estimated covariance matrices and expected returns contain errors

These challenges motivate the exploration of quantum computing approaches, which we'll cover in the next notebooks.